In [1]:
!pip install transformers==4.36.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 85.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.0
    Uninstalling transformers-4.44.0:
      Successfully uninstalled transformers-4.44.0


In [2]:
!pip install --upgrade --quiet evaluate jiwer

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2024.7.1 requires dask==2024.7.1, but you have dask 2024.8.1 which is incompatible.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.2 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
osmnx 1.9.4 requires shapely<2.1,>=2.0, but you have shapely 1.8.5.post1 which is incompatible.
pointpats 2.5.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
rapids-dask-dependency 24.8.0a0 requires dask==2024.7.1, but you have dask 2024.8.1 which is incompatible.
spaghetti 1.7.6 requires shapely>=2.0.1, b

In [ ]:
#asr libraries

import ssl
import logging
import sys
import os
import base64
import io

import numpy as np
import time
import torch

import json
import traceback

from transformers import pipeline

from pydub import AudioSegment
from io import BytesIO
import glob
import uuid
import asyncio

#asr variables
DATASET_PATH = None
unique_name = None
device = None
pipe = None
model = "maddi99/uiu_bn_new_1"
CHUNK_LENGTH_S = 20.1

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [4]:
from transformers import WhisperFeatureExtractor,WhisperTokenizer,WhisperProcessor

feature_extractor = WhisperFeatureExtractor.from_pretrained(model)
tokenizer = WhisperTokenizer.from_pretrained(model, language="bn", task="transcribe")
processor = WhisperProcessor.from_pretrained(model, language="bn", task="transcribe")

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.5k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.18M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/255k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
from datasets import Dataset, load_dataset, Audio

# Create your dataset object (example)
dataset1 = load_dataset('maddi99/uiu_faculty_new_3')
dataset1 = dataset1.cast_column("audio", Audio(sampling_rate=16000))

Generating train split:   0%|          | 0/125 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1 [00:00<?, ? examples/s]

In [6]:
dataset1

DatasetDict({
    train: Dataset({
        features: ['transcription', 'audio'],
        num_rows: 125
    })
    test: Dataset({
        features: ['transcription', 'audio'],
        num_rows: 1
    })
})

In [7]:
# Create your dataset object (example)
dataset2 = load_dataset('maddi99/merged_audio_dataset_1')
dataset2 = dataset2.cast_column("audio", Audio(sampling_rate=16000))

Generating train split:   0%|          | 0/3845 [00:00<?, ? examples/s]

In [8]:
dataset2

DatasetDict({
    train: Dataset({
        features: ['text', 'audio', 'file_name', 'language', 'dataset_name', 'source_dataset'],
        num_rows: 3845
    })
})

In [9]:
from datasets import DatasetDict, concatenate_datasets

dataset2_modified = DatasetDict({
    "train": dataset2["train"].rename_column("text", "transcription")
})

In [10]:
dataset2_modified

DatasetDict({
    train: Dataset({
        features: ['transcription', 'audio', 'file_name', 'language', 'dataset_name', 'source_dataset'],
        num_rows: 3845
    })
})

In [11]:
# Step 2: Keep only the columns we need ('audio' and 'transcription') from dataset2
dataset2_modified["train"] = dataset2_modified["train"].remove_columns(
    [col for col in dataset2_modified["train"].column_names 
     if col not in ["audio", "transcription"]]
)

In [12]:
dataset2_modified

DatasetDict({
    train: Dataset({
        features: ['transcription', 'audio'],
        num_rows: 3845
    })
})

In [13]:
merged_train = concatenate_datasets([dataset1["train"], dataset2_modified["train"]])

In [14]:
merged_train

Dataset({
    features: ['transcription', 'audio'],
    num_rows: 3970
})

In [15]:
final_dataset = DatasetDict({
    "train": merged_train,
    "test": dataset1["test"]
})

In [16]:
final_dataset

DatasetDict({
    train: Dataset({
        features: ['transcription', 'audio'],
        num_rows: 3970
    })
    test: Dataset({
        features: ['transcription', 'audio'],
        num_rows: 1
    })
})

In [17]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["transcription"]).input_ids
    return batch

In [18]:
final_dataset = final_dataset.map(prepare_dataset,remove_columns=final_dataset.column_names["train"],
                      num_proc=4)

/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/3970 [00:00<?, ? examples/s]

num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [19]:
final_dataset

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 3970
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 1
    })
})

In [20]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(model)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

In [21]:
tokenizer.get_decoder_prompt_ids(language="bn", task="transcribe")

[(1, 50302), (2, 50359), (3, 50363)]

In [22]:
model.config.forced_decoder_ids = tokenizer.get_decoder_prompt_ids(language="bn", task="transcribe")

In [23]:
# from datasets import load_dataset, concatenate_datasets
# # Step 2: Combine the selected splits
# combined_splits = concatenate_datasets([dataset["class_lecture"], 
#                                         dataset["kid_cartoon"], 
#                                         dataset["drama_series"],
#                                         dataset["parliament_speech"],
#                                         dataset["biography"],
#                                         dataset["kid_voice"]
#                                        ])

In [24]:
shuffled_dataset = final_dataset.shuffle(seed=42)  # Seed is optional for reproducibility

In [25]:
shuffled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 3970
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 1
    })
})

In [26]:
validation_dataset = load_dataset('maddi99/uiu_en_100')
# validation_dataset = validation_dataset.cast_column("audio", Audio(sampling_rate=16000))
validation_dataset = validation_dataset.map(prepare_dataset,remove_columns=
                                            validation_dataset.column_names["train"],
                      num_proc=4)

Generating train split:   0%|          | 0/95 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/24 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/95 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/24 [00:00<?, ? examples/s]

In [27]:
validation_dataset

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 95
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 24
    })
})

In [28]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [29]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [30]:
# import evaluate

# metric = evaluate.load("wer")

In [31]:
from datasets import load_metric
cer_metric = load_metric("cer",trust_remote_code=True)  # uses Hugging Face's metric implementation

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    # Replace with your label decoding logic if different
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    return {"cer": cer}


/tmp/ipykernel_23/1227161948.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  cer_metric = load_metric("cer",trust_remote_code=True)  # uses Hugging Face's metric implementation


In [32]:
import wandb


In [ ]:
!wandb login 

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [34]:
wandb.init(project="uiu_bn_new_8", entity="maddi99")

wandb: Currently logged in as: maddi99. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.19.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.7
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250503_073017-4labwc2m
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run pretty-deluge-1
wandb: ⭐️ View project at https://wandb.ai/maddi99/uiu_bn_new_8
wandb: 🚀 View run at https://wandb.ai/maddi99/uiu_bn_new_8/runs/4labwc2m


In [35]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./uiu_bn_new_8",  # change to a repo name of your choice
    per_device_train_batch_size=3,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=50,
    max_steps=9000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=3,
    predict_with_generate=True,
#     generation_max_length=225,
    save_steps=9000,
    eval_steps=500,
    logging_steps=500,
    report_to="wandb",
    load_best_model_at_end=True,
    metric_for_best_model="cer",
    greater_is_better=False,
#     push_to_hub=True,
)

In [36]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=shuffled_dataset["train"],
    eval_dataset=validation_dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:451: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [37]:
# rm -rf "/kaggle/working/uiu_bn_new_3"

In [38]:
processor.save_pretrained(training_args.output_dir)

In [39]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss,Cer
500,0.663200,1.186916,1.000000
1000,0.372700,1.353508,1.000000
1500,0.295900,1.189376,1.000000
2000,0.245800,1.123520,1.000000
2500,0.216800,1.237286,1.000000
3000,0.178700,1.317277,1.000000
3500,0.162100,1.327294,1.000000
4000,0.166400,1.454500,1.000000
4500,0.115000,1.343260,1.000000
5000,0.114200,1.388794,1.000000


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


TrainOutput(global_step=9000, training_loss=0.1706394502851698, metrics={'train_runtime': 32230.3704, 'train_samples_per_second': 0.838, 'train_steps_per_second': 0.279, 'total_flos': 2.754408524414976e+19, 'train_loss': 0.1706394502851698, 'epoch': 6.8})

In [40]:
kwargs = {
    "dataset_tags": "maddi99/uiu_bn_new_5",
    "dataset": "uiu transcription",  # a 'pretty' name for the training dataset
    "language": "bn",
    "model_name": "Whisper Medium - maddi",  # a 'pretty' name for your model
    "finetuned_from": "maddi99/uiu_bn_new_1",
    "tasks": "automatic-speech-recognition",
}

In [41]:
trainer.push_to_hub(**kwargs)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/maddi99/uiu_bn_new_8/commit/d64f250794e4444e5b4d6965345229a8a5b79440', commit_message='End of training', commit_description='', oid='d64f250794e4444e5b4d6965345229a8a5b79440', pr_url=None, pr_revision=None, pr_num=None)